In [1]:
import h5py
import pandas as pd
import json
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ssp
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from collections import Counter
import pickle
from sklearn.utils import shuffle
from os.path import join as pjoin
from tqdm import tqdm
import os
import joblib
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import importlib 
from sklearn.preprocessing import normalize as sk_normalize
import pytorch_lightning as pl
from os.path import join as pjoin
from scipy import sparse

In [2]:
torch.cuda.set_device(1)

In [3]:
fname = "procseed_data_with_aug"

In [45]:
te_ds = joblib.load('%s/te_ds' % fname)

In [5]:
tr_ds = joblib.load('%s/tr_ds' % fname)
val_ds = joblib.load("%s/val_ds" % fname) 
id2idx = joblib.load('%s/id2idx' % fname)
idx2id = {y:x for (x,y) in id2idx.items()}
n_items = len(id2idx)

In [54]:
val_csv = joblib.load('%s/val_csv' % fname)
te_csv = joblib.load('%s/te_csv' % fname)

In [55]:
val_csv.shape, te_csv.shape

((40808, 13), (40810, 12))

In [7]:
def flatten(x):
    ret = []
    for k in x:
        ret.extend(k)
    return ret

def get_t(tr_csv, date='2021-04-01 00:00:00.000', tonly=False):
    tg = tr_csv.date_session_end >= date    
    ret = set()
    ret = set(tr_csv[tg].target_item.tolist())
    if tonly:
        ret |= set(flatten(tr_csv[tg].item_ids.tolist()))
#         ret |= set(tr_csv[tg].target_item.tolist())
    print(len(ret))
    r = np.zeros(len(id2idx), dtype=bool)
    rr = [id2idx[x] for x in ret]
    r[rr] = True
    return r

In [21]:
idmap = joblib.load('../processed/indices')[0]

In [11]:
item_feats = pd.read_csv('../data/item_features.csv', dtype=str)
item_feats['feat_id'] = item_feats['feature_category_id'] + ':' + item_feats['feature_value_id']
d = item_feats.groupby("item_id")['feat_id'].apply(list).reset_index()
item_to_feats = {x:y for (x, y) in zip(d.item_id, d.feat_id)}

In [24]:
def new_logit_map(logits):
    idmap = joblib.load('../processed/indices')[0]
    new_map = np.ones(shape=(logits.shape[0], np.max(list(idmap.values())) + 1)) * -100000.0
    idmap = [idmap[idx2id[x]] for x in range(logits.shape[1])]
    new_map[:, idmap] = logits
    return new_map

In [13]:
def validate_models(models, val_ds, r=None):
    hr = []
    for model in models:
        model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    for input in DataLoader(val_ds, batch_size=250, shuffle=True):
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
        if r is not None:
            ret[:, ~r] = -100000.0
        ret[ret_mat['item_bow'].bool()] = -10000.0
        top_rec = (-ret).argsort(-1)[:, :100].detach().cpu()
        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        hitst = mrr[:, :5].sum(-1)
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0)).sum(-1)
        tot_mrr.extend(mrr.tolist())
        hr.extend(hitst)
    for model in models:
        model = model.train()
    return np.mean(tot_mrr), np.mean(hr)


In [14]:
def get_res(models, val_ds):
    for model in models:
        model = model.eval()
    s = 0
    rets = []
    ii = 0
    for input in DataLoader(val_ds, batch_size=250, shuffle=False):
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
        ret[ret_mat['item_bow'].bool()] = -10000.0
        rets.append(ret.detach().cpu().numpy())
        ii += 1
#         if ii == 2:
#             break
    return np.vstack(rets)

In [15]:
models = torch.load('val_mlp_no_shuffle')# + torch.load('no_shuffle_mlp')
models = [model.cuda().eval() for model in models]

In [16]:
val_score = validate_models(models, val_ds)

In [15]:
val_score

(0.19207685421335882, 0.26742306)

In [20]:
te_score = validate_models(models, te_ds)

In [21]:
te_score

(0.19129616796707752, 0.26785102)

In [18]:
val_logits = get_res(models, val_ds)

In [19]:
te_logits = get_res(models, te_ds)

In [26]:
val_new_logits = new_logit_map(val_logits)

In [27]:
te_new_logits = new_logit_map(te_logits)

In [57]:
val_new_logits.shape, te_new_logits.shape

((40808, 23618), (40810, 23618))

In [56]:
rm ../logits/test/mlp-noshuffle*

rm: cannot remove '../logits/test/mlp-noshuffle*': No such file or directory


In [59]:
len(te_csv.session_id.tolist())

40810

In [61]:
!ls -al "../logits/test/"

total 7530472
drwxr-xr-x 2 root root          0 Jul  6 15:55 .
drwxr-xr-x 7 root root          0 Jul  6 15:55 ..
-rw-r--r-- 1 root root 7711203064 Jul  6 15:56 mlp-noshuffle-1657090556


In [64]:
ls '/data/project/rw/recsys-challenge-2022-2105/logits/test'

mlp-noshuffle-1657090556


In [58]:
import os
import time
os.makedirs("../logits/val", exist_ok=True)
os.makedirs("../logits/test", exist_ok=True)
# joblib.dump(
#     (val_csv.session_id.tolist(), val_new_logits),
#     "../logits/val/mlp-noshuffle-%d" % int(time.time()))

joblib.dump(
    (te_csv.session_id.tolist(), te_new_logits),
    "../logits/test/mlp-noshuffle-%d" % int(time.time()))

['../logits/test/mlp-noshuffle-1657090556']

In [42]:
a, b = joblib.load('../logits/test/mlp-noshuffle-1657085878')

In [44]:
b.shape

(40810, 23618)